In [2]:
import numpy as np
import csv
import pandas as pd
import pyprind
import scipy.sparse as spr
import graphlab as gl
from sklearn.cluster import KMeans
from math import isnan
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import mean_absolute_error
sns.set_style("whitegrid")
sns.set_context("poster")
pd.set_option('display.width', 500)
pd.set_option('display.max_columns', 100)
pd.set_option('display.notebook_repr_html', True)
%matplotlib inline

In [5]:
train_file = 'data/train.csv'
test_file  = 'data/test.csv'
profiles_file = 'data/profiles.csv'
artists_file = 'data/artists.csv'

# Experimentation

### User Profile Matrix
#### Load the User data into a DataFrame

In [3]:
profiles = pd.read_csv(profiles_file, header=0)
user_ids = profiles.user

In [4]:
profile_data = pd.DataFrame({'sex': profiles.sex.apply(lambda s: -1 if type(s) == float and np.isnan(s) else s).values, \
                                 'age': profiles.age.apply(lambda s: int(s) if not isnan(s) else -1).values, \
                                 'country': profiles.country.values}, \
                                index=user_ids.values)

In [5]:
profile_data.head()

,age,country,sex
fa40b43298ba3f8aa52e8e8863faf2e2171e0b5d,25,Sweden,f
5909125332c108365a26ccf0ee62636eee08215c,29,Iceland,m
d1867cbda35e0d48e9a8390d9f5e079c9d99ea96,30,United States,m
63268cce0d68127729890c1691f62d5be5abd87c,21,Germany,m
02871cd952d607ba69b64e2e107773012c708113,24,Netherlands,m


In [6]:
columns = list(set(profile_data.sex)) + list(set(profile_data.country)) + [15, 20, 25, 30, 35, 40, 45, 50]

In [7]:
#This was helpful: http://stackoverflow.com/questions/2272149/round-to-5-or-other-number-in-python
def custom_round(x, base=5):
    rounded = int(base * round(float(x)/base))
    if rounded <15:
        return 15
    elif rounded > 50:
        return 50
    else:
        return rounded

#### Create a Matrix with 1 and 0 with a auser has a certaint characteristic

In [8]:
# User Profile Matrix
profile_matrix = np.zeros((len(profile_data.index.values), len(columns)))
loop_size = len(profile_data.index.values)
mybar = pyprind.ProgBar(loop_size)

for i, user in enumerate(profile_data.index.values):
    profile = profile_data.ix[user]

    #Sex indices
    if profile['sex'] == 'm':
        profile_matrix[i, 0] = 1  
    elif profile['sex'] == 'f':
        profile_matrix[i, 1] = 1
        
    # Country indices
    country = profile['country']
    country_col = columns.index(country)
    profile_matrix[i, country_col] = 1
    
    # Age indices
    age = custom_round(profile['age'])
    profile_matrix[i, columns.index(age)] = 1
    
    mybar.update()

In [9]:
user_pos_by_id = {}
for i, user_id in enumerate(user_ids):
    user_pos_by_id[user_id] = i

In [10]:
train_data = pd.read_csv(train_file, header=0)

In [11]:
train = pd.DataFrame({'user': train_data.user.values, \
                        'artist': train_data.artist.values, \
                        'plays': train_data.plays.apply(lambda s: int(s) if not isnan(s) else 0).values}, \
                        index=train_data.user.values)

In [12]:
train.ix["44ce793a6cd9d20f13f4a576a818ef983314bb5d"].head()

,artist,plays,user
44ce793a6cd9d20f13f4a576a818ef983314bb5d,a3a92047-be1c-4f3e-8960-c4f8570984df,81,44ce793a6cd9d20f13f4a576a818ef983314bb5d
44ce793a6cd9d20f13f4a576a818ef983314bb5d,5441c29d-3602-4898-b1a1-b77fa23b8e50,70,44ce793a6cd9d20f13f4a576a818ef983314bb5d
44ce793a6cd9d20f13f4a576a818ef983314bb5d,f4857fb9-e255-4dc6-bd01-e4ca7cc68544,21,44ce793a6cd9d20f13f4a576a818ef983314bb5d
44ce793a6cd9d20f13f4a576a818ef983314bb5d,6ffb8ea9-2370-44d8-b678-e9237bbd347b,56,44ce793a6cd9d20f13f4a576a818ef983314bb5d
44ce793a6cd9d20f13f4a576a818ef983314bb5d,63011a8d-0117-4f7e-9991-1ef1f337ff70,13,44ce793a6cd9d20f13f4a576a818ef983314bb5d


### Train/Validate Split

In [13]:
train.head()

,artist,plays,user
eb1c57ddc9e0e2d005169d3a1a96e8dd95e3af03,5a8e07d5-d932-4484-a7f7-e700793a9c94,554,eb1c57ddc9e0e2d005169d3a1a96e8dd95e3af03
44ce793a6cd9d20f13f4a576a818ef983314bb5d,a3a92047-be1c-4f3e-8960-c4f8570984df,81,44ce793a6cd9d20f13f4a576a818ef983314bb5d
da9cf3f557161d54b76f24db64be9cc76db008e3,eeb1195b-f213-4ce1-b28c-8565211f8e43,708,da9cf3f557161d54b76f24db64be9cc76db008e3
8fa49ab25d425edcf05d44bfc1d5aea895287d81,a1419808-65d3-4d40-998c-1a0bac65eabc,265,8fa49ab25d425edcf05d44bfc1d5aea895287d81
b85fcaef67d2669cd99b334b5e8c8705263db2cf,a3cb23fc-acd3-4ce0-8f36-1e5aa6a18432,220,b85fcaef67d2669cd99b334b5e8c8705263db2cf


In [14]:
trainlist=[]
validatelist=[]
take=4
i = 0
loop_size = len(train.groupby('user'))
mybar = pyprind.ProgBar(loop_size)
for k, v in train.groupby('user'):
    if v.artist.count() > 10:
        validatelist.append(v[-take:]) 
        trainlist.append(v[:-take])
    else:
        trainlist.append(v)
    mybar.update()
train_df=pd.concat(trainlist)
validate_df=pd.concat(validatelist)

0%                          100%
[##############################] | ETA: 00:00:00
Total time elapsed: 00:02:09


In [15]:
print len(set(train_df.user.values))
print len(set(validate_df.user.values))

233286
224957


In [16]:
# CALCULATE THE NUMBER OF SONGS WE ARE ESTIMATING IN OUR TRIAN_TEST SAMPLE
num_songs_estimating = 0

for user, user_data in train_df.iteritems():
    for artist, plays in user_data.iteritems():
        num_songs_estimating += 1
        
print num_songs_estimating

9764928


In [17]:
KM = KMeans(n_clusters=20, \
            init='k-means++', \
            n_init=10, \
            max_iter=300, \
            tol=0.0001, \
            precompute_distances='auto', \
            verbose=0, \
            random_state=37)
# Calls fit and then predict
predict = KM.fit_predict(profile_matrix)

In [18]:
print "The objective function: %f" % KM.score(profile_matrix) 

The objective function: -206859.457500


In [19]:
# Examine the predicted clusters
print predict[1:10]

[ 4  4  9 17  9 19  1  7 10]


### Global-Median

In [20]:
global_median = np.median(np.array(train_df.plays.values))
print global_median

118.0


### User-Medians

In [21]:
user_medians = {}

uuids = list(set(train_df.user))
loop_size = len(uuids)
mybar = pyprind.ProgBar(loop_size)
for k, user in enumerate(uuids):
    user_medians[user] = np.median(np.array(train_df.ix[user].plays.values))
    mybar.update()

0%                          100%
[##############################] | ETA: 00:00:00
Total time elapsed: 00:01:45


In [22]:
medians_df = pd.DataFrame.from_dict(user_medians, orient='index')
medians_df.rename(columns={0:"user_median"}, inplace=True)
medians_df["global_median"] = global_median
medians_df["user"] = medians_df.index.values

In [23]:
medians_df.head()

,user_median,global_median,user
f283c15ed4180e686384dc1de2a5cbf5f95ae269,4.0,118,f283c15ed4180e686384dc1de2a5cbf5f95ae269
5909125332c108365a26ccf0ee62636eee08215c,415.0,118,5909125332c108365a26ccf0ee62636eee08215c
0eae120959c04371c23af09abaf71305ab2a1b3c,237.5,118,0eae120959c04371c23af09abaf71305ab2a1b3c
734f7337c7d33e99fa60a6361a5df8e3fb939ecf,242.0,118,734f7337c7d33e99fa60a6361a5df8e3fb939ecf
02871cd952d607ba69b64e2e107773012c708113,172.0,118,02871cd952d607ba69b64e2e107773012c708113


### Create a Dataframe with actual plays of validation set and the user_medians of the train set

In [24]:
validate_df.head()

,artist,plays,user
00001411dc427966b17297bf4d69e7e193135d89,e105c272-b5d7-4135-82ef-d60bded54345,1244,00001411dc427966b17297bf4d69e7e193135d89
00001411dc427966b17297bf4d69e7e193135d89,9fa07bb4-8312-4cd7-a19a-4fdbc5797148,2641,00001411dc427966b17297bf4d69e7e193135d89
00001411dc427966b17297bf4d69e7e193135d89,ffb18e19-64a4-4a65-b4ce-979e00c3c69d,622,00001411dc427966b17297bf4d69e7e193135d89
00001411dc427966b17297bf4d69e7e193135d89,451f9db1-f75f-44f9-b218-f8bdf22035a1,2427,00001411dc427966b17297bf4d69e7e193135d89
000063d3fe1cf2ba248b9e3c3f0334845a27a6bf,50eec634-7c42-41ee-9b1f-b41d9ca28b26,8,000063d3fe1cf2ba248b9e3c3f0334845a27a6bf


In [25]:
final_df = validate_df
final_df["user_median"] = 0

In [26]:
final_df.head()

,artist,plays,user,user_median
00001411dc427966b17297bf4d69e7e193135d89,e105c272-b5d7-4135-82ef-d60bded54345,1244,00001411dc427966b17297bf4d69e7e193135d89,0
00001411dc427966b17297bf4d69e7e193135d89,9fa07bb4-8312-4cd7-a19a-4fdbc5797148,2641,00001411dc427966b17297bf4d69e7e193135d89,0
00001411dc427966b17297bf4d69e7e193135d89,ffb18e19-64a4-4a65-b4ce-979e00c3c69d,622,00001411dc427966b17297bf4d69e7e193135d89,0
00001411dc427966b17297bf4d69e7e193135d89,451f9db1-f75f-44f9-b218-f8bdf22035a1,2427,00001411dc427966b17297bf4d69e7e193135d89,0
000063d3fe1cf2ba248b9e3c3f0334845a27a6bf,50eec634-7c42-41ee-9b1f-b41d9ca28b26,8,000063d3fe1cf2ba248b9e3c3f0334845a27a6bf,0


In [54]:
"""loop_size = len(medians_df.user_median)
mybar = pyprind.ProgBar(loop_size)
test = []
for k, user in enumerate(list(set(medians_df.user))):
    #test.append(medians_df.ix[user].user_median)
    if user in final_df.user:
        final_df.loc[user, "user_median"] = medians_df.ix[user].user_median
    mybar.update()"""


0%                          100%
[##############################] | ETA: 00:00:00
Total time elapsed: 00:36:04


In [55]:
#final_df.to_csv("user_medians_validation.csv", sep='\t', encoding='utf-8')

In [58]:
final_df = pd.read_csv("user_medians_validation.csv", sep='\t', encoding='utf-8')

In [59]:
final_df.head()

,Unnamed: 0,artist,plays,user,user_median
0,00001411dc427966b17297bf4d69e7e193135d89,e105c272-b5d7-4135-82ef-d60bded54345,1244,00001411dc427966b17297bf4d69e7e193135d89,854.5
1,00001411dc427966b17297bf4d69e7e193135d89,9fa07bb4-8312-4cd7-a19a-4fdbc5797148,2641,00001411dc427966b17297bf4d69e7e193135d89,854.5
2,00001411dc427966b17297bf4d69e7e193135d89,ffb18e19-64a4-4a65-b4ce-979e00c3c69d,622,00001411dc427966b17297bf4d69e7e193135d89,854.5
3,00001411dc427966b17297bf4d69e7e193135d89,451f9db1-f75f-44f9-b218-f8bdf22035a1,2427,00001411dc427966b17297bf4d69e7e193135d89,854.5
4,000063d3fe1cf2ba248b9e3c3f0334845a27a6bf,50eec634-7c42-41ee-9b1f-b41d9ca28b26,8,000063d3fe1cf2ba248b9e3c3f0334845a27a6bf,4.0


### Validation Performance
### MAE for User_median: 137

In [60]:
mean_absolute_error(final_df.plays, final_df.user_median)

137.26561187249118

In [62]:
final_df_submit = final_df[["user", "user_median"]]

In [63]:
final_df_submit.head()

,user,user_median
0,00001411dc427966b17297bf4d69e7e193135d89,854.5
1,00001411dc427966b17297bf4d69e7e193135d89,854.5
2,00001411dc427966b17297bf4d69e7e193135d89,854.5
3,00001411dc427966b17297bf4d69e7e193135d89,854.5
4,000063d3fe1cf2ba248b9e3c3f0334845a27a6bf,4.0


### Making the prediction

In [65]:
test_df = pd.read_csv(test_file, header=0)

In [66]:
test_df.head()

,Id,user,artist
0,1,306e19cce2522fa2d39ff5dfc870992100ec22d2,4ac4e32b-bd18-402e-adad-ae00e72f8d85
1,2,9450d351278df4938bdea4ed86aec940a4e927ac,1f574ab1-a46d-4586-9331-f0ded23e0411
2,3,801909d6955f59033c88595d3d7f8a6a5dcd53cc,3eb72791-6322-466b-87d3-24d74901eb2d
3,4,e3ed47445c127fbeff47fb58f6bbf2f3b4535d82,61604b45-8a91-4e33-a1b6-45d7b1fec4e5
4,5,a73f46652103f3a5f7429159310f6928f79644aa,5dfdca28-9ddc-4853-933c-8bc97d87beec


In [ ]:
test_df["artist"] = 0
test_df.rename(columns={"artist": "plays"}, inplace=True)
test_df.head()

In [ ]:
loop_size = len(medians_df.user_median)
mybar = pyprind.ProgBar(loop_size)
i = 0
for k, user in enumerate(list(set(medians_df.user))):
    if user in test_df["user"].values:
        i += 1
        test_df.loc[user, "plays"] = medians_df.ix[user].user_median
    mybar.update()

0%                          100%
[                              ]

In [89]:
#test_df.user.values
#medians_df.ix["734f7337c7d33e99fa60a6361a5df8e3fb939ecf"].user_median

array(['306e19cce2522fa2d39ff5dfc870992100ec22d2',
       '9450d351278df4938bdea4ed86aec940a4e927ac',
       '801909d6955f59033c88595d3d7f8a6a5dcd53cc', ...,
       'f7fefb72eea0fa7976eb2fc5c0f1fe9f5d1149bd',
       '6af6ded3c4a0894e65aed78d48f979a0ac3ce1ae',
       'de29005de66c93b31a34991b0c72d9f70b9fc313'], dtype=object)

## GraphLab

In [14]:
gl.canvas.set_target('ipynb')

In [55]:
# The below will download a 118 MB file.
sf = gl.SFrame.read_csv(train_file, header=True, verbose=False, column_type_hints={"plays":int})

In [60]:
#sf.rename({'user':'user_id', "artist": "item_id"})
sf.rename({"artist": "item_id"})

user_id,item_id,plays
eb1c57ddc9e0e2d005169d3a1a96e8dd95e3af03 ...,5a8e07d5-d932-4484-a7f7-e700793a9c94 ...,554
44ce793a6cd9d20f13f4a576a818ef983314bb5d ...,a3a92047-be1c-4f3e-8960-c4f8570984df ...,81
da9cf3f557161d54b76f24db64be9cc76db008e3 ...,eeb1195b-f213-4ce1-b28c-8565211f8e43 ...,708
8fa49ab25d425edcf05d44bfc1d5aea895287d81 ...,a1419808-65d3-4d40-998c-1a0bac65eabc ...,265
b85fcaef67d2669cd99b334b5e8c8705263db2cf ...,a3cb23fc-acd3-4ce0-8f36-1e5aa6a18432 ...,220
feed7a0dc74c5251283a1505adf453a2061d08f7 ...,1cc5adcd-1422-4b5c-a3cd-3ecd4f43f506 ...,2113
cbb86d88a8d2d0bab8956807c6c45cd0c752324b ...,9c9f1380-2516-4fc9-a3e6-f9f61941d090 ...,127
5641e1e6f04868a61dc29f7227e34f4640163e9b ...,832a43c7-aa7d-439b-a6b4-4f1afa671c24 ...,305
9f748976d303db79f61bf570d9549d6335b11b0e ...,2fddb92d-24b2-46a5-bf28-3aed46f4684c ...,705
056d5d2467dc63c4520963323e2ebf9576b58229 ...,847e8284-8582-4b0e-9c26-b042a4f49e57 ...,7


In [71]:
sf_test = gl.SFrame.read_csv(test_file, header=True, verbose=False, column_type_hints={"plays":int})
sf_test.rename({'user':'user_id', "artist": "item_id"})

Id,user_id,item_id
1,306e19cce2522fa2d39ff5dfc870992100ec22d2 ...,4ac4e32b-bd18-402e-adad-ae00e72f8d85 ...
2,9450d351278df4938bdea4ed86aec940a4e927ac ...,1f574ab1-a46d-4586-9331-f0ded23e0411 ...
3,801909d6955f59033c88595d3d7f8a6a5dcd53cc ...,3eb72791-6322-466b-87d3-24d74901eb2d ...
4,e3ed47445c127fbeff47fb58f6bbf2f3b4535d82 ...,61604b45-8a91-4e33-a1b6-45d7b1fec4e5 ...
5,a73f46652103f3a5f7429159310f6928f79644aa ...,5dfdca28-9ddc-4853-933c-8bc97d87beec ...
6,55f1c89e3a102de38a0bdfcb1fe660b028c5c0af ...,ef58d4c9-0d40-42ba-bfab-9186c1483edd ...
7,7ad7619f38bf52de421b795f3159949422b5bbee ...,a3cb23fc-acd3-4ce0-8f36-1e5aa6a18432 ...
8,2a1ab291185cfc6fb996556481e178666e9731f9 ...,5dfdca28-9ddc-4853-933c-8bc97d87beec ...
9,3f407fff902ab403f06668f6be3d10bb9e9a02cd ...,000fc734-b7e1-4a01-92d1-f544261b43f5 ...
10,2eea5a2b68e6e9e0d9dd6b3c1c0e1e5840fc9ce3 ...,84783313-2a3f-4f17-ab9a-425a5d527879 ...


In [38]:
#http://blog.dato.com/choosing-a-recommender-model

#### Item Similarity Models

In [ ]:
item_sim_model = gl.item_similarity_recommender.create(train_set, user_id="user", item_id="artist", target="plays")

result_ismodel = item_sim_model.predict(sf_test)

scaled_result_ismodel = (result_ismodel - result_ismodel.min())/(result_ismodel.max() - result_ismodel.min()) * sf['plays'].max()
print scaled_result_ismodel

In [37]:

with open("data/item_sim_results.csv", 'w') as soln_fh:
    soln_csv = csv.writer(soln_fh,
                          delimiter=',',
                          quotechar='"',
                          quoting=csv.QUOTE_MINIMAL)
    soln_csv.writerow(['Id', 'plays'])
    loop_size = 4154804
    mybar = pyprind.ProgBar(loop_size)
    for i in range(4154804):
        soln_csv.writerow([i + 1, scaled_result_ismodel[i]])
        mybar.update()

#### Popularity Model

In [ ]:
#Popularity Model
popularity_model = gl.popularity_recommender.create(train_set, user_id="user_id", item_id="", target="plays")

result_pm = popularity_model.predict(sf_test)

scaled_result_pm = (result_pm - result_pm.min())/(result_pm.max() - result_pm.min()) * sf['plays'].max()
print scaled_result_pm

with open("data/popularity_model_results.csv", 'w') as soln_fh:
    soln_csv = csv.writer(soln_fh,
                          delimiter=',',
                          quotechar='"',
                          quoting=csv.QUOTE_MINIMAL)
    soln_csv.writerow(['Id', 'plays'])
    loop_size = 4154804
    mybar = pyprind.ProgBar(loop_size)
    for i in range(4154804):
        soln_csv.writerow([i + 1, scaled_result_pm[i]])
        mybar.update()

#### Factorization Recommender

In [69]:
user_info = gl.SFrame.read_csv(profiles_file, header=True, verbose=False).dropna()
user_info.rename({"user": "user_id"})

item_info = gl.SFrame.read_csv(artists_file, header=True, verbose=False).dropna()
item_info.rename({"artist": "item_id"})

factorization_model = gl.factorization_recommender.create(sf, target='plays', \
        user_data=user_info, \
        item_data=item_info)

Recsys training: model = factorization_recommender

Preparing data set.

Data has 4154804 observations with 233286 users and 2000 items.

Data prepared in: 6.62665s

Training factorization_recommender for recommendations.

...

+---------+--------------+-------------------+-----------------------+-------------+

Optimization Complete: Maximum number of passes through the data reached.

Computing final objective value and training RMSE.

Final objective value: 406490

Final training RMSE: 637.565

In [72]:
result_fm = factorization_model.predict(sf_test)

scaled_result_fm = (result_fm - result_fm.min())/(result_fm.max() - result_fm.min()) * sf['plays'].max()
print scaled_result_fm

with open("data/factorization_model_results.csv", 'w') as soln_fh:
    soln_csv = csv.writer(soln_fh,
                          delimiter=',',
                          quotechar='"',
                          quoting=csv.QUOTE_MINIMAL)
    soln_csv.writerow(['Id', 'plays'])
    loop_size = 4154804
    mybar = pyprind.ProgBar(loop_size)
    for i in range(4154804):
        soln_csv.writerow([i + 1, scaled_result_fm[i]])
        mybar.update()

0%                          100%
[##############################] | ETA: 00:00:00

[126589.9086052624, 126149.35035344078, 126860.41042363497, 127179.35514811601, 126248.15224840365, 126137.11054576185, 127073.72321182169, 126630.45022026588, 129744.80045072273, 125434.86314587752, 125596.65664816876, 127086.09997808782, 125890.059747164, 126611.75345949495, 126743.90989168786, 126770.33517681311, 126874.08498783663, 129291.66683791726, 126008.32091449168, 126324.1751028491, 125302.60357262075, 126149.35424323128, 125733.72624397627, 130474.46634822941, 126494.22123822037, 127454.49392436605, 126455.12525925816, 126225.3174548953, 125607.45568596211, 126517.42473622785, 126818.30153189495, 126161.4465024317, 125781.80658399346, 131347.85751924713, 126114.4221331824, 127680.90788069298, 125361.68791648406, 125775.86851346258, 127304.20717705789, 127349.34867343778, 126325.37573317853, 125967.9506260165, 127292.8541788464, 127098.96415240053, 129005.76603681965, 129055.98589724903, 127962.53760870126, 126322.35132891864, 126965.34332670047, 126413.52815834564, 126612.5


Total time elapsed: 00:13:35


In [73]:
result_fm

dtype: float
Rows: 4154804
[238.5622514834668, 169.05476199149925, 281.23970042545506, 331.56005418962434, 184.6428746815276, 167.1236704727044, 314.8943563582103, 244.95855859425325, 736.3139356022425, 56.32911931767012, 81.85550570103541, 316.8470560242131, 128.14612160936994, 242.00874458567304, 262.8592491079762, 267.028403176867, 283.39715548077595, 664.822413878832, 146.8043508250275, 196.63710639701014, 35.46234208594916, 169.05537568916793, 103.4811668953599, 851.4343413462902, 223.46552220490236, 374.96908243947706, 217.2972951074471, 181.04019747988386, 83.55928494666429, 227.126370429318, 274.5961218792954, 170.96318827883405, 111.06686915992145, 989.2304838334235, 163.54408799425764, 410.69072793166606, 44.7841613759943, 110.13001149508375, 351.25813282927174, 358.3801696366687, 196.82653151766462, 140.4350741394253, 349.46695447629884, 318.87665473713423, 619.715445967392, 627.6387033497008, 455.1238426882783, 196.3493670418111, 297.795110809962, 210.7344625135293, 242.126

In [74]:
#Alternating Least Squares Model
als_model = gl.factorization_recommender.create(sf, target='plays', solver = 'als')

result_als = als_model.predict(sf_test)

scaled_result_als = (result_als - result_als.min())/(result_als.max() - result_als.min()) * sf['plays'].max()
print scaled_result_als

with open("data/popularity_model_results.csv", 'w') as soln_fh:
    soln_csv = csv.writer(soln_fh,
                          delimiter=',',
                          quotechar='"',
                          quoting=csv.QUOTE_MINIMAL)
    soln_csv.writerow(['Id', 'plays'])
    loop_size = 4154804
    mybar = pyprind.ProgBar(loop_size)
    for i in range(4154804):
        soln_csv.writerow([i + 1, scaled_result_als[i]])
        mybar.update()

Recsys training: model = factorization_recommender

Preparing data set.

Data has 4154804 observations with 233286 users and 2000 items.

Data prepared in: 6.72592s

Training factorization_recommender for recommendations.

...

+---------+--------------+------------------------+

Optimization Complete: Iteration limit reached.

0%                          100%
[##############################] | ETA: 00:00:00

[238610.74922568726, 238971.32364234547, 239000.67325308928, 239049.14187597352, 238938.0717390984, 238986.11719675062, 238975.0791579878, 239115.74242785058, 240008.71945606527, 238969.08956375797, 238936.6838972174, 239012.25881250718, 238939.90615107285, 238871.59027367856, 239008.40998857038, 238893.25522247853, 238858.9214956692, 239047.97964556175, 238956.34769816638, 238953.09674008968, 238918.19577006417, 238594.51482704643, 238943.06511964195, 241260.71105421483, 238959.1120856944, 238987.75135660073, 238983.97654442728, 238922.45926387564, 238952.70221674556, 238466.55695625683, 238962.27583898715, 238931.70414139103, 238934.59769585365, 239879.0395133392, 238945.54365598774, 238276.81095184348, 238896.76504193922, 238890.6875938351, 238887.79957415623, 238998.18829899066, 238906.95031342466, 238950.70245362085, 238876.4419917816, 238934.35783324344, 238992.37013572478, 239077.50567702332, 239037.63006147073, 238972.4472467253, 239215.34009128466, 238929.293489601, 239042.916


Total time elapsed: 00:13:20


In [75]:
result_als

dtype: float
Rows: 4154804
[-1140.0237311242417, 195.0041390539809, 303.67116175905903, 483.1263848425551, 71.88886653200825, 249.77731906191548, 208.9089470984145, 729.7151299597426, 4035.9652825476333, 186.7324563147231, 66.750377762477, 346.5667306067153, 78.68078242556294, -174.25896061642925, 332.3164483191176, -94.0444220422105, -221.165149581273, 478.8232232214614, 139.55566035525044, 127.51897822634419, -1.7019690392808116, -1200.1316412804917, 90.37687312380513, 8671.467968094508, 149.79081355349263, 255.82780085818013, 241.85154734865864, 14.083629715602001, 126.05825434939106, -1673.8960455773667, 161.50461970583638, 48.31281672732075, 59.02621089235981, 3555.8249016882583, 99.55366145388325, -2376.4312018273667, -81.0493048547105, -103.55107487424175, -114.24397648556987, 294.47061167971333, -43.33830631955425, 118.65412532107075, -156.29545963986675, 58.13811885134419, 272.928844559352, 588.143444168727, 440.50385676638325, 199.16428767458638, 1098.4757805945083, 39.387371

### Ridge Regression

In [20]:
# Design Matrix
from sklearn.linear_model import Ridge
features=np.concatenate([uuids,uiids])
features.shape

(235286,)

In [19]:
def getmats(indf):
    stvals=indf[['user', 'artist', 'plays']].values
    designm=np.zeros((stvals.shape[0], features.shape[0]))
    ratings=np.zeros(stvals.shape[0])
    loop_size = len(stvals)
    mybar = pyprind.ProgBar(loop_size)
    for i, row in enumerate(stvals):
        designm[i,:]=np.concatenate([1*(row[0]==uuids), 1*(row[1]==uiids)])
        ratings[i]=row[2]
        mybar.update()
    return designm, ratings

In [ ]:
#Carrying out the ridge regression
designm, ratings = getmats(train_df)

0%                          100%
[                              ]

In [ ]:
validatedm, validaterats = getmats(validate_df)

In [122]:
#alphas=[0.01, 0.1, 10, 100, 1000]
alphas=[7,8,9,10,11,12,13]
vdict={}
rdict={}
loop_size = len(alphas)
mybar = pyprind.ProgBar(loop_size)
for a in alphas:
    regr=Ridge(alpha=a).fit(designm, ratings)
    vpreds=regr.predict(validatedm)
    mae=mean_absolute_error(validaterats, vpreds)
    vdict[a]=mae
    rdict[a]=regr
    mybar.update()

0%   100%
[#######] | ETA: 00:00:00
Total time elapsed: 00:06:58


In [123]:
minerroralpha=min(vdict, key=vdict.get)
print minerroralpha
regr=rdict[minerroralpha] 

11


In [124]:
regr.intercept_, ybar

(252.07751787456544, 252.7676107730693)

In [ ]:
testdm, _ = getmats(test_df)

In [164]:
predictions['baseline_r']=regr.predict(testdm)

In [242]:
np.savetxt("baseline_r.csv", predictions['baseline_r'], delimiter=",")